In [1]:
import unittest
import numpy as np
from scipy.ndimage import binary_dilation, sobel
from scipy.spatial.distance import cdist
from scipy.stats import wasserstein_distance

In [2]:
voxel_base = np.load("voxel_grid_base.npy")
voxel_identical = np.load("voxel_grid_identical.npy")
voxel_similar = np.load("voxel_grid_similiar.npy")
voxel_different = np.load("voxel_grid_different.npy")
voxel_growth = np.load("voxel_grid_growth.npy")
voxel_missing = np.load("voxel_grid_missing.npy")

## Hausdorff Distance 
Measure focusing on the maximum distance between the nearest points in each set.
- Hausdorff Distance = 0: The grids are identical in terms of occupied voxel positions.<br>
- Hausdorff Distance > 0: Some voxels in one grid are not directly aligned with voxels in the other grid.

#### Definition


$$
d_H(A, B) = \max\{ h(A, B), h(B, A) \}
$$


$$
h(A, B) = \max_{a \in A} \min_{b \in B} d(a, b)
$$

d(a, b) denotes the distance between points a and b.

$$ 
d(a, b) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2 + (z_2 - z_1)^2} 
$$

#### Time complexity O(k1 x k2) where 
k1- number of occupied voxels in first grid<br>
k2- number of occupied voxels in 2nd grid


#### Properties

1. **Symmetry**
2. **Non-negativity**
3. **Identity of indiscernibles**: If \( d_H(A, B) = 0 \), then the sets \( A \) and \( B \) are equivalent in the sense that they contain the same points.
4. **Triangle Inequality**

In [3]:
def hausdorff_distance(voxel1, voxel2):
    """
    Calculate the Hausdorff distance between two 3D voxel grids.

    The Hausdorff distance is the maximum distance of a set to the nearest point in the other set.
    It measures how far the two voxel grids are from each other.

    Parameters:
    voxel1 (numpy.ndarray): A 3D numpy array representing the first voxel grid.
    voxel2 (numpy.ndarray): A 3D numpy array representing the second voxel grid.

    Returns:
    float: The Hausdorff distance between the two voxel grids.
    """
    # Get coordinates of occupied voxels in both grids
    coords1 = np.array(np.nonzero(voxel1)).T
    coords2 = np.array(np.nonzero(voxel2)).T

    # Compute distances from each point in voxel1 to the closest point in voxel2
    dists_1_to_2 = cdist(coords1, coords2, 'euclidean').min(axis=1)
    dists_2_to_1 = cdist(coords2, coords1, 'euclidean').min(axis=1)

    # Hausdorff distance is the max of these minimum distances
    hausdorff_dist = max(dists_1_to_2.max(), dists_2_to_1.max())
    return hausdorff_dist

In [4]:
print(f"Hausdorf distance between base and identical: {hausdorff_distance(voxel_base, voxel_identical)}")
print(f"Hausdorf distance between base and similar: {hausdorff_distance(voxel_base, voxel_similar)}")
print(f"Hausdorf distance between base and different: {hausdorff_distance(voxel_base, voxel_different)}")
print(f"Hausdorf distance between base and growth: {hausdorff_distance(voxel_base, voxel_growth)}")
print(f"Hausdorf distance between base and missing: {hausdorff_distance(voxel_base, voxel_missing)}")

Hausdorf distance between base and identical: 0.0
Hausdorf distance between base and similar: 6.164414002968976
Hausdorf distance between base and different: 16.0312195418814
Hausdorf distance between base and growth: 8.48528137423857
Hausdorf distance between base and missing: 15.524174696260024


## Total Variation Distance (TVD)

A measure that quantifies the difference between two probability distributions represented by voxel grids. It is calculated as the normalized absolute difference between the corresponding voxels in the two grids.

- **Total Variation Distance = 0**: The grids are identical, meaning the corresponding voxel values are the same in both grids.<br>
- **Total Variation Distance > 0**: There are differences between the corresponding voxel values in the two grids.

#### Definition

The Total Variation Distance is defined as:

$$
\text{TVD}(A, B) = \frac{1}{2} \sum_{i} |A_i - B_i|
$$

We are using normalization so its [0,1], so the used formula is:

$$
   \text{TVD} = \frac{1}{2} \cdot \frac{\sum_{i} |A_i - B_i|}{\text{size of the grid}}
$$

Where \( A_i \) and \( B_i \) represent the values of the corresponding voxels in grids \( A \) and \( B \).

#### Time Complexity O(n)


where n is the total number of voxels in the voxel grids.

#### Properties

1. **Range**: The Total Variation Distance ranges from 0 to 1 (or 0 to 0.5 when normalized), where 0 indicates identical distributions and higher values indicate greater divergence.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If TVD(A, B) = 0 , then the voxel grids \( A \) and \( B \) are equivalent in terms of their voxel values.
5. **Triangle Inequality**

In [5]:
def total_variation_distance(voxel1, voxel2):
    """
    Calculate the Total Variation Distance (TVD) between two voxel arrays.

    The Total Variation Distance is a measure of the difference between two 
    probability distributions. It is used to measure the difference between two voxel arrays.

    Parameters:
    voxel1 (np.ndarray): The first voxel array (3D).
    voxel2 (np.ndarray): The second voxel array (3D).

    Returns:
    float: The Total Variation Distance between the two voxel arrays.
    """
    # Compute absolute difference and normalize
    abs_difference = np.abs(voxel1.astype(float) - voxel2.astype(float))
    tvd = np.sum(abs_difference) / (2 * voxel1.size)
    return tvd

In [6]:
print("Total Variation Distance between base and identical: ", total_variation_distance(voxel_base, voxel_identical))
print("Total Variation Distance between base and similar: ", total_variation_distance(voxel_base, voxel_similar))
print("Total Variation Distance between base and different: ", total_variation_distance(voxel_base, voxel_different))
print("Total Variation Distance between base and growth: ", total_variation_distance(voxel_base, voxel_growth))
print("Total Variation Distance between base and missing: ", total_variation_distance(voxel_base, voxel_missing))

Total Variation Distance between base and identical:  0.0
Total Variation Distance between base and similar:  0.05550540842166918
Total Variation Distance between base and different:  0.052648299054894925
Total Variation Distance between base and growth:  0.10180856324675211
Total Variation Distance between base and missing:  0.03904314774187322


## 

## Wasserstein Distance

A measure that quantifies the difference between two probability distributions represented by voxel grids. The Wasserstein distance (also known as the Earth Mover's Distance) takes into account the distance that needs to be "moved" to transform one distribution into another.

- **Wasserstein Distance = 0**: The grids are identical, meaning the distributions represented by the voxel values are the same in both grids.<br>
- **Wasserstein Distance > 0**: There are differences between the distributions represented by the corresponding voxel values in the two grids.

#### Definition

The Wasserstein Distance between two probability distributions \( P \) and \( Q \) is defined as:

$$
W(P, Q) = \inf_{\gamma \in \Gamma(P, Q)} \int_{X \times Y} c(x, y) d\gamma(x, y)
$$

Where:
- \( \Gamma(P, Q) \) is the set of all possible couplings of \( P \) and \( Q \),
- \( c(x, y) \) is the cost function representing the distance between points \( x \) and \( y \).

#### Time Complexity O(n log n)


#### Properties

1. **Range**: The Wasserstein Distance is always non-negative and ranges from 0 to infinity, where 0 indicates identical distributions.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If \( W(P, Q) = 0 \), then the distributions \( P \) and \( Q \) are equivalent in terms of their voxel values.
5. **Triangle Inequality**


In [7]:
def wasserstein_dst_voxel(voxel1, voxel2):
    """
    Compute the Wasserstein distance between two voxel grids.
    The function flattens the 3D voxel grids into 1D arrays and then computes the 
    Wasserstein distance between these arrays.
    Parameters:
    voxel1 (numpy.ndarray): The first voxel grid.
    voxel2 (numpy.ndarray): The second voxel grid.
    Returns:
    float: The Wasserstein distance between the two voxel grids.
    """
    # Flatten the voxel grids to 1D arrays
    voxel1_flat = voxel1.ravel().astype(float)
    voxel2_flat = voxel2.ravel().astype(float)
    
    # Compute Wasserstein distance
    wasserstein_dist = wasserstein_distance(voxel1_flat, voxel2_flat)
    return wasserstein_dist

In [8]:
print("Wasserstein distance between base and identical: ", wasserstein_dst_voxel(voxel_base, voxel_identical))
print("Wasserstein distance between base and similar: ", wasserstein_dst_voxel(voxel_base, voxel_similar))
print("Wasserstein distance between base and different: ", wasserstein_dst_voxel(voxel_base, voxel_different))
print("Wasserstein distance between base and growth: ", wasserstein_dst_voxel(voxel_base, voxel_growth))
print("Wasserstein distance between base and missing: ", wasserstein_dst_voxel(voxel_base, voxel_missing))

Wasserstein distance between base and identical:  0.0
Wasserstein distance between base and similar:  0.0067370468176728895
Wasserstein distance between base and different:  0.0038859269250881194
Wasserstein distance between base and growth:  0.09967560652231103
Wasserstein distance between base and missing:  0.02769267304711749


## Gradient Magnitude Similarity Deviation (GMSD)

Measure based on voxels gradient magnitudes. It quantifies how similar the structures within the voxel grids are by analyzing the gradient information, which highlights changes in intensity.
Gradient Magnitude is a measure of the rate of change of intensity or value in an image or a function at a given point.

- **GMSD = 1**: The grids are identical in terms of their gradient magnitudes.<br>
- **GMSD < 1**: There are differences in the gradient magnitudes between the two grids, indicating dissimilarity.

#### Definition

The GMSD is defined as:

$$
\text{GMSD}(A, B) = 1 - \frac{1}{N} \sum_{i} \left| \frac{G_A(i) - G_B(i)}{G_A(i) + G_B(i) + \epsilon} \right|
$$

Where:
- \( G_A(i) \) and \( G_B(i) \) are the gradient magnitudes at voxel \( i \) for grids \( A \) and \( B \).
- \( N \) is the total number of voxels.
- epsilon is a small constant added to prevent division by zero.

### Time Complexity O(n)

### Properties

1. **Range**: The GMSD ranges from 0 to 1, where 1 indicates identical gradient magnitudes and lower values indicate greater divergence.
2. **Symmetry**
3. **Non-negativity**
4. **Identity of indiscernibles**: If GMSD(A, B) = 1 , then the voxel grids \( A \) and \( B \) are equivalent in terms of their gradient magnitudes.


In [9]:
def gradient_magnitude_similarity_deviation(voxel1, voxel2):
    """
    Compute the Gradient Magnitude Similarity Deviation (GMSD) between two voxel grids.
    Parameters:
    voxel1 (ndarray): The first 3D voxel grid.
    voxel2 (ndarray): The second 3D voxel grid.
    Returns:
    float: The GMSD value, which is a measure of the similarity between the gradient magnitudes of the two voxel grids.
           A value closer to 1 indicates higher similarity, while a value closer to 0 indicates lower similarity.
    Notes:
    - The function computes the gradient magnitude of each voxel grid using the Sobel operator along each axis.
    - The absolute difference between the gradient magnitudes of the two voxel grids is then computed.
    - The GMSD is calculated as 1 minus the mean of the normalized absolute differences.
    - A small constant (1e-6) is added to the denominator to avoid division by zero.
    """
    # Compute the gradient magnitude of each voxel grid
    gradient_magnitude1 = np.sqrt(sum(sobel(voxel1, axis=i)**2 for i in range(3)))
    gradient_magnitude2 = np.sqrt(sum(sobel(voxel2, axis=i)**2 for i in range(3)))
    
    # Compute the absolute difference of the gradient magnitudes
    gmsd = 1- np.mean(np.abs((gradient_magnitude1 - gradient_magnitude2) / (gradient_magnitude1 + gradient_magnitude2 + 1e-6)))
    return gmsd

In [10]:
print("Gradient Magnitude Similarity Deviation between base and identical: ", gradient_magnitude_similarity_deviation(voxel_base, voxel_identical))
print("Gradient Magnitude Similarity Deviation between base and similar: ", gradient_magnitude_similarity_deviation(voxel_base, voxel_similar))
print("Gradient Magnitude Similarity Deviation between base and different: ", gradient_magnitude_similarity_deviation(voxel_base, voxel_different))
print("Gradient Magnitude Similarity Deviation between base and growth: ", gradient_magnitude_similarity_deviation(voxel_base, voxel_growth))
print("Gradient Magnitude Similarity Deviation between base and missing: ", gradient_magnitude_similarity_deviation(voxel_base, voxel_missing))

Gradient Magnitude Similarity Deviation between base and identical:  1.0
Gradient Magnitude Similarity Deviation between base and similar:  0.8569276114253821
Gradient Magnitude Similarity Deviation between base and different:  0.8302531577720123
Gradient Magnitude Similarity Deviation between base and growth:  0.8398348357589751
Gradient Magnitude Similarity Deviation between base and missing:  0.8531313361705859
